zzzzzzzzzzzzhotfix

In [1]:
#@title ## Start Stable Diffusion
from IPython.display import clear_output
from google.colab import drive
import os
import time
import shutil
# Variables
model_path = "/content/gdrive/MyDrive/stable-diffusion-webui/models/Stable-diffusion"
ext_path = "/content/gdrive/MyDrive/stable-diffusion-webui/extensions"
vae_path = "/content/gdrive/MyDrive/stable-diffusion-webui/models/VAE"
lora_path = "/content/gdrive/MyDrive/stable-diffusion-webui/models/Lora"
controlnet_path = "/content/gdrive/MyDrive/stable-diffusion-webui/extensions/sd-webui-controlnet/models"
embd_path = "/content/gdrive/MyDrive/stable-diffusion-webui/embeddings"

if not os.path.exists("/content/gdrive/MyDrive/"):
  print("Google drive not connected, using Colab storage instead...")
  !mkdir -p /content/gdrive/MyDrive/

# Cloning Repo
%cd /content/gdrive/MyDrive
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui
%cd /content/gdrive/MyDrive/stable-diffusion-webui

clear_output()

# Installing Requirements
!apt -y update -qq
!wget http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
!wget http://launchpadlibrarian.net/367274642/google-perftools_2.5-2.2ubuntu3_all.deb
!wget http://launchpadlibrarian.net/367274648/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
!wget http://launchpadlibrarian.net/367274647/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
!apt install -qq libunwind8-dev
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *.deb

# Temporary bug fix for https://github.com/AUTOMATIC1111/stable-diffusion-webui/issues/9341
%pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118
%pip install  torchtext torchaudio torchdata --extra-index-url https://download.pytorch.org/whl/cu118

%pip install --upgrade fastapi==0.90.1
%pip install huggingface-hub==0.11.1
clear_output()

Model_Link = "" #@param {type:"string"}
Model_Filename = "" #@param {type:"string"}

#Lora
lora1 = "https://civitai.com/api/download/models/29870" #@param {type:"string"}
lora2 = "" #@param {type:"string"}
lora3 = "" #@param {type:"string"}
lora4 = "" #@param {type:"string"}
if lora1 != '':
  !wget $lora1 -P $lora_path --content-disposition
if lora2 != '':
  !wget $lora2 -P $lora_path --content-disposition
if lora3 != '':
  !wget $lora3 -P $lora_path --content-disposition
if lora4 != '':
  !wget $lora4 -P $lora_path --content-disposition

safetensors = True #@param {type:"boolean"}
if Model_Link != "":
  time.sleep(1)
  %cd $model_path
  if Model_Filename is None:
    Model_Filename = "model"
  if not safetensors:
    modelname = Model_Filename + ".ckpt"
  else:
    modelname = Model_Filename + ".safetensors"
  if os.path.exists(f"{model_path}/{modelname}"):
    print("Model already exist! Using", modelname)
  else:
    time.sleep(1)
    !wget $Model_Link -O "$model_path/$modelname"
elif Model_Name == "SD v1.5":
  if os.path.exists(f"{model_path}/v1-5-pruned-emaonly.ckpt"):
    print("Model already exist! Using", Model_Name)
  else:
    time.sleep(1)
    %cd $model_path
    !wget https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt -O "$model_path/v1-5-pruned-emaonly.ckpt"
elif Model_Name == "ChilloutMix":
  if os.path.exists(f"{model_path}/chilloutmix_NiPrunedFp32Fix.safetensors"):
    print("Model already exist! Using", Model_Name)
  else:
    time.sleep(1)
    %cd $model_path
    !wget https://civitai.com/api/download/models/11745 -O "$model_path/chilloutmix_NiPrunedFp32Fix.safetensors"
elif Model_Name == "Grapefruit v4.1":
  if os.path.exists(f"{model_path}/grapefruitHentaiModel_grapefruitv41.safetensors"):
    print("Model already exist! Using", Model_Name)
  else:
    time.sleep(1)
    %cd $model_path
    !wget https://civitai.com/api/download/models/29179 -O "$model_path/grapefruitHentaiModel_grapefruitv41.safetensors"
elif Model_Name == "Counterfeit v2.5":
  if os.path.exists(f"{model_path}/CounterfeitV25_25.safetensors"):
    print("Model already exist! Using", Model_Name)
  else:
    time.sleep(1)
    %cd $model_path
    !wget https://civitai.com/api/download/models/7425 -O "$model_path/CounterfeitV25_25.safetensors"




clear_output()


# Extension
!git clone https://github.com/SlavyanDesu/sd-webui-tunnels.git $ext_path/sd-webui-tunnels
!git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete.git $ext_path/a1111-sd-webui-tagcomplete
!git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111.git $ext_path/ultimate-upscale-for-automatic1111
!git clone https://github.com/AlUlkesh/stable-diffusion-webui-images-browser.git $ext_path/stable-diffusion-webui-images-browser
!git clone https://github.com/camenduru/sd-civitai-browser.git $ext_path/sd-civitai-browser
!git clone https://github.com/catppuccin/stable-diffusion-webui.git $ext_path/catppuccin-theme
!git clone https://github.com/Mikubill/sd-webui-controlnet.git $ext_path/sd-webui-controlnet
!git clone https://github.com/fkunn1326/openpose-editor.git $ext_path/openpose-editor
!git clone https://github.com/KohakuBlueleaf/a1111-sd-webui-locon.git $ext_path/a1111-sd-webui-locon
!git clone https://github.com/camenduru/stable-diffusion-webui-huggingface.git $ext_path/stable-diffusion-webui-huggingface
!git clone https://github.com/kohya-ss/sd-webui-additional-networks.git $ext_path/sd-webui-additional-networks
!git clone https://github.com/butaixianran/Stable-Diffusion-Webui-Civitai-Helper $ext_path/Stable-Diffusion-Webui-Civitai-Helper
clear_output()

# ControlNet
ControlNet = False #@param {type:"boolean"}
#@markdown <small><font color=gray> **HINT**: Check `ControlNet` to download ControlNet models.</small><br>
if ControlNet == True:
  if os.path.exists(f"{controlnet_path}"):
    print("ControlNet already exist!")
  else:
    time.sleep(1)
    !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_canny-fp16.safetensors -P $controlnet_path
    !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_depth-fp16.safetensors -P $controlnet_path
    !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_hed-fp16.safetensors -P $controlnet_path
    !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_mlsd-fp16.safetensors -P $controlnet_path
    !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_normal-fp16.safetensors -P $controlnet_path
    !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_openpose-fp16.safetensors -P $controlnet_path
    !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_scribble-fp16.safetensors -P $controlnet_path
    !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_seg-fp16.safetensors -P $controlnet_path
  clear_output()

# VAE
VAE = True #@param {type:"boolean"}
#@markdown <small><font color=gray> **HINT**: Check `VAE` to download VAE(s).</small><br>

if VAE == True:
  time.sleep(1)
  !wget https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime.ckpt -P $vae_path
  !wget https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt -P $vae_path
  !wget https://huggingface.co/gsdf/Counterfeit-V2.5/resolve/main/Counterfeit-V2.5.vae.pt -P $vae_path
  !wget https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt -P $vae_path
  !wget https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -P $vae_path

shutil.rmtree(embd_path)
%cd /content/gdrive/MyDrive/stable-diffusion-webui
!git clone https://huggingface.co/SlavyanDesu/embeddings
clear_output()

# Run Stable Diffusion
!sed -i -e 's/checkout {commithash}/checkout --force {commithash}/g' /content/gdrive/MyDrive/stable-diffusion-webui/launch.py
!COMMANDLINE_ARGS="--share --api --listen --xformers --enable-insecure-extension-access --reinstall-xformers --disable-safe-unpickle --remotemoe --no-half-vae" REQS_FILE="requirements.txt" python launch.py

💥